#  Pacman por Caminhos Desconhecidos
## Teste de Avaliação Contínua nº 2
### Sistemas Inteligentes edição 2022/23
### Entrega: 29 de Março (5m antes da meia-noite)

<img src="manyghosts.gif" alt="Drawing" style="width: 150px;"/>



## Introdução
Imaginem uma variante do jogo do Pacman que habita um mundo discreto 2D, numa grelha com obstáculos e sem gravidade, e que deseja encontrar o melhor caminho até à pastilha. O Pacman pode mover-se nas quatro direcções ortogonais para as quatro células vizinhas, desde que não seja impedido por obstáculos ou pela sua fronteira. Todas as acções possuem o mesmo custo.

Há um pequeno problema: o Pacman é lançado para este mundo sem conhecer qualquer mapa a não ser a sua posição, a posição da pastilha e as suas 8 células vizinhas. Não sabe se o mundo é um espaço aberto ou se está cercado. Como é um optimista, na verdade sofre da doença *optimus extremus*, assume que todas as células que desconhece são livres para circular.

O Pacman não gosta de improvisar e não arrisca dar um passo sem um plano de acções que o levem à pastilha. Assim, ele planeia sempre as suas ações de acordo com o modelo que possui no momento, assumindo de forma optimista que o resto do mapa é uma maravilha, sem obstáculos. Notem que no seu plano todas as posições a percorrer atè à posição da pastilha serão células livres de obstáculos. À medida que se move e executa criteriosamente o plano acabadinho de fazer, e ele não gosta nada de se afastar do seu plano, as suas ações levam-no a explorar o mundo para zonas desconhecidas e inesperadas, permitindo que vá actualizando e aperfeiçoando o seu modelo --- notem que ele em cada momento percepciona sempre as suas 8 células vizinhas, ortogonal e diagonalmente adjacentes.

No entanto, ele pode ficar em apuros quando a próxima célula indicada pelo plano for diferente da do mundo real. Se afinal a próxima posição indicada pelo plano é um obstáculo, o pacman interrompe imediatamente a sua execução, e teimoso como é, não arrisca avançar sem replanear. Logo que pára, ele irá delinear um novo plano a partir da sua nova posição e do seu modelo, agora talvez mais refinado, não perdendo nunca o seu optimismo. As células vizinhas que foi observando até parar, que não fazem parte da rota traçada e que são diferentes do mundo conhecido até então, não O obrigam a replanear, servem antes para actualizar o seu mapa, o qual estará mais afinado para o planeamento que se seguirá. 

O que vai acontecer é que o Pacman até conseguir chegar à pastilha vai entrar num ciclo de planeamento-execução com actualização do modelo do mundo, em que cada planeamento é feito a partir de diferentes posições iniciais e em que o modelo do mundo que resulta da execução online dos seus planos se vai aproximando cada vez mais do mundo real. Notem que pode acontecer que o Pacman pode ir parar a becos sem saída, pode optar por explorar zonas do espaço que afinal não fazem parte do melhor caminho para atingir a pastilha, pois a sua atitude optimista leva-no a pensar que está a aproximar-se da sua pastilhinha, mas só assim é que consegue mapear correctamente o espaço.

Para planear as suas acções, ele usa sempre o algoritmo A* (ele prefere usar a versão em grafo) e pode ter duas estratégias diferentes: de planeamento em planeamento pode atualizar a heurística ou não. A heurística inicial é a distância de Manhattan da sua posição à pastilha. Na primeira estratégia, os valores heurísticos das posições ao longo do seu modelo e que vai explorando, nunca mudam. Na segunda estratégia, chamada Adaptative-A*, após terminar o seu planeamento, atualiza a heurística de algumas das células do seu modelo de acordo com o custo do melhor caminho até essas células e também do custo da solução que resultou do seu plano. Note que sendo a distância de Manhatan consistente, o A* em quaisquer das versões (árvore ou grafo) devolve a solução óptima (de acordo com o modelo), a solução com o menor caminho. As células onde a heurística é actualizada são aquelas para as quais tem a garantia que encontrou o caminho óptimo partindo da posição com que começou nesse plano.

## Mundo 2D
Considerem que cada célula do mundo é dada por um par de coordenadas $(x,y)$ e que a célula de topo à esquerda é a célula $(0,0)$ e que as coordenadas crescem para a direita e para baixo. O Pacman ocupa uma dessas posições e a pastilha também. Podem existir obstáculos e a própria fronteira do mundo, se existir é feita de obstáculo. O Pacman e a pastilha não podem ocupar posições com obstáculos.

As funções seguintes: ***line*** e ***quadro*** permitem ajudar a construir os obstáculos e a fronteira do mundo.

```python
def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)
```

Os nossos testes automáticos vão basear-se muito na apresentação do mundo real e do modelo do mundo do Pacman. Em qualquer deles o importante está nas posições do Pacman e da pastilha e dos obstáculos. Também importante é saber qual foi o caminho feito pelo Pacman, se existiu algum, e assim essa função permite também apresentar os passos que deu o Pacman. Assim fornecemos a função ***display***  que podem usar de modo a que a apresentação em texto seja uniforme.

``` python
def display(pacman,pastilha,obstaculos,path=[]):
    """ print the state please"""
    pacmanX,pacmanY=pacman
    osXs={x for (x,_) in obstaculos | {pastilha, pacman}}
    minX=min(osXs)
    maxX=max(osXs)
    osYs={y for (_,y) in obstaculos | {pastilha, pacman}}
    minY=min(osYs)
    maxY=max(osYs)
    output=""
    for j in range(minY,maxY+1):
        for i in range(minX,maxX+1):
            if pacman ==(i,j):
                ch = '@'
            elif pastilha==(i,j):
                ch = "*"
            elif (i,j) in obstaculos:
                ch = "#"
            elif (i,j) in path:
                ch = '+'
            else:
                ch = "."
            output += ch + " "
        output += "\n"
    print(output)
```

Todas estas funções estão em `p2_aux.py`.

Vamos mostrar um exemplo de um mapa real.

In [17]:
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
fronteira = quadro(0,0,10)
display((1,1),(3,3),fronteira | l | c)

# # # # # # # # # # 
# @ . . . . . . . # 
# . # # # # # # . # 
# . # * . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 



In [18]:
obstaculos = [(0,0),(0,1),(0,2), (1,0), (2,0), (2,2)]
display((1,1),(3,3),set(obstaculos))

# # # . 
# @ . . 
# . # . 
. . . * 



Inicialmente o Pacman apenas conhece a sua posição, a da pastilha e as 8 à sua volta. Eis assim, o seu modelo inicial. 
```python
# # # . 
# @ . . 
# . # . 
. . . *
```
Todas as células que aqui não estão ele assume que são livres de serem percorridas. Um optimista que pode chegar a ser irritante.
Neste caso os obstáculos são formados pelas células: $(0,0),(1,0),(2,0),(0,1),(0,2),(2,2)$

Imaginemos agora que o Pacman construiu um plano para atingir a pastilha e que esse plano foi: $S \longrightarrow  S \longrightarrow E \longrightarrow E$. O positivo é que mesmo que tenha conseguido dar dois passos até esbarrar no obstáculo em $(2,3)$ que desconhecia, tendo executado o plano apenas parcialmente, actualizou o seu modelo com mais 4 obstáculos, dando dois passos .

O seu novo modelo corresponde agora a:
```python
# # # . 
# + . . 
# + # . 
# @ # * 
# . # .
```
que é obtido através de:
```
display((1,3),(7,8),{(0,0),(1,0),(2,0),(0,1),(0,2),(2,2),(0,3),(0,4),(2,3),(2,4)})
```
Quando omitimos o parâmetro `path` não apresentamos nenhum rasto.

Se quisermos que o seu rasto, formado por +'s, apareça inscrito no seu novo modelo, poderemos invocar a função ***display*** com os inputs seguintes:
```python
display((1,3),(7,8),{(0,0),(1,0),(2,0),(0,1),(0,2),(2,2),(0,3),(0,4),(2,3),(2,4)},[(1,1),(1,2)])
# # # . 
# + . . 
# + # . 
# @ # * 
# . # . 
```

Também poderemos fazer:
```python
display((1,3),(7,8),{(0,0),(1,0),(2,0),(0,1),(0,2),(2,2),(0,3),(0,4),(2,3),(2,4)},[(1,1),(1,2),(1,3)])
# # # . 
# + . . 
# + # . 
# @ # * 
# . # . 
```

## O Pacman planeia
Para planear o Pacman vai formular o problema de encontrar o melhor caminho entre a sua posição actual e a pastilha como um processo de procura num grafo de estados e usar um algoritmo de procura, o A*, para encontrar a solução. Na sua formulação as acções serão 'N', 'S', 'O' e 'E' e correspondem aos movimentos no seu mapa modelo para norte, sul, oeste e este respectivamente desde que não esbarre num obstáculo,para representar os movimentos para norte, para sul, para oeste e para este, respectivamente.

## O Pacman planeia com estratégia adaptativa
<img src="ta1b.jpg" alt="Drawing" style="width: 350px;"/>

O Adaptative A* é um método pensado para jogos em que o mundo vai sendo explorado à medida que o jogador se movimenta no mapa.

Vamos tratar de todo o processo numa única função ***planeia_adaptativo_online*** que recebe apenas a posição do pacman, a posição da pastilha e os obstáculos. A função deve tratar de toda a lógica de procura. Esta versão faz uso do A* adaptative, e devolve a contabilidade dos estados expandidos.

A ideia do Adaptative A* é encontrar melhores heurísticas sempre que possível, isto é, após cada execução do A*, de modo a melhorar o desempenho da execução dos A* seguintes. 

Assumamos que *s* é um estado que foi expandido durante a última procura A*. Podemos obter uma estimativa admissível da distância do *estado s* ao *objectivo* da seguinte maneira: A distância do *estado inicial* ao objectivo através do *estado s* é igual à distância do *estado inicial* ao *estado s* mais a distância de *s* ao *objectivo*. 
$$ d(I, s, g) = d(I, s) + d(d, g) $$

Assim, a distância do *estado s* ao *objectivo* é maior ou igual à distância do *estado inicial* ao *objectivo* (= o valor de f do estado objectivo que estava para ser expandido quando o A* terminou) menos a distância do *inicial* ao *estado s* (=o valor do estado s quando o A* terminou).
$$ d(s, g) \ge d(I, g) - d(I, s)$$

O nosso A* Adaptativo obtém assim uma heurística melhor informada calculando e afectando esta diferença a cada estado que tenha sido expandido durante a execução do A* e que pertence à lista fechada quando o A* termina. (Os estados na lista aberta não são actualizados porque a distância do estado inicial a estes estados pode ser ainda menor do que os seus valores-g no momento em que o A* acabou.)

Reparem no Exemplo 6, abaixo explicado. Também podem consultar os dois artigos disponibilizados com este enunciado, que se encontram nesta pasta. 

### Algoritmo de procura: A*
O algoritmo que terão de usar é o A*. Como vai ser usada uma heurística consistente, a solução das duas versões do A*, árvore ou grafo, é sempre a mesma, mas lembramos a preferência do Pacman pela versão em grafo por ser mais eficiente durante o processo de procura da solução no caso de uma grelha 2D em que há em geral múltiplos caminhos diferentes entre quaisquer 2 pontos.

#### Desempate na fila de prioridades
O A* usa uma fila de prioridades para guardar a árvore de procura (a fronteira de nós) em que dá preferência aos nós com menor valor de $f \space (f=g+h)$. No entanto, em caso de empate dá preferência aos nós com estados menores numa ordem de estados. Neste caso, é natural que os estados sejam apenas as posições do mapa, representadas por tuplos com as 2 cooordenadas $(x,y)$. Já existe uma ordenação natural dos tuplos e é essa que vai ser respeitada pelo A*: $(0,1)$ precede $(1,0)$ e precede também $(0,3)$. No fundo as posições mais a norte e mais à esquerda são mais prioritárias em caso de empate.

## Objectivo
O objectivo deste projecto é implementar duas funções: a função ***planear_online*** e a função **planear_adapt_online**, em que a primeira usa o A* com a distância de Manhatan como função heurística que nunca muda ao longo de cada planeamento, e a segunda utiliza o A* mas em que as heurísticas de algumas posições do mundo vão sendo progressivamente melhoradas, de plano para plano. Notem que evolução do planeamento online é sempre igual para ambas as funções, dados os mesmos inputs: o número de planos, os planos e suas execuções serão rigorosamente os mesmos, mas o processo de procura executado pelo A* é que será diferente devido à diferença nas heurísticas.

Ambas as funções recebem como inputs:
* a posição inicial do Pacman
* a posição da pastilha
* os obstáculos do mundo

devolvem como output:
* None

Elas não devolvem nada mas ao serem executadas vão apresentar no ecrã a evolução de todo o processo de planeamento online.

### A função ***planear_online***
Esta função irá começar por mostrar o mundo e o modelo inicial do Pacman e depois em cada iteração irá mostrar o plano gerado pelo A* bem como o número de estados expandidos e o novo modelo do mundo com a sua nova posição e as marcas das posições por onde andou. No final deve indicar o total de expandidos acumulados ao longo dos vários planeamentos.

#### Exemplo 1
Vejamos o exemplo seguinte, em que basta apenas uma iteração:
```python
pacman=(1,1)
pastilha=(1,6)
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
fronteira = quadro(0,0,10)
obstaculos=fronteira | l | c
planeia_online(pacman,pastilha,obstaculos)
```

dará como output:
```python
MUNDO
# # # # # # # # # # 
# @ . . . . . . . # 
# . # # # # # # . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# * # . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 

MODELO
# # # 
# @ . 
# . # 
. . . 
. . . 
. . . 
. * . 

ITERAÇÃO: 1
['S', 'S', 'S', 'S', 'S']
Expandidos 5
# # # 
# + . 
# + # 
# + # 
# + # 
# + # 
# @ # 
# . . 

FIM: total de expandidos: 5
```
#### Exemplo 2

```python
pacman=(1,2)
pastilha=(3,6)
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
fronteira = quadro(0,0,10)
obstaculos=fronteira | l | c
planeia_online(pacman,pastilha,obstaculos)

MUNDO
# # # # # # # # # # 
# . . . . . . . . # 
# @ # # # # # # . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # * . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 

MODELO
# . . . 
# @ # . 
# . # . 
. . . . 
. . . . 
. . . * 

ITERAÇÃO: 1
['S', 'S', 'S', 'S', 'E', 'E']
Expandidos 10
# . . . 
# + # . 
# + # . 
# + # . 
# + # . 
# @ # * 
# . . . 

ITERAÇÃO: 2
['S', 'E', 'E', 'N']
Expandidos 5
# . . . 
# . # . 
# . # . 
# . # . 
# . # . 
# + # @ 
# + + + 
# . . . 

FIM: total de expandidos: 15
```
#### Exemplo 3
```python
pacman=(1,1)
pastilha=(3,3)
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
fronteira = quadro(0,0,10)
obstaculos=fronteira | l | c
planeia_online(pacman,pastilha,obstaculos)

MUNDO
# # # # # # # # # # 
# @ . . . . . . . # 
# . # # # # # # . # 
# . # * . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 

MODELO
# # # . 
# @ . . 
# . # . 
. . . * 

ITERAÇÃO: 1
['S', 'S', 'E', 'E']
Expandidos 7
# # # . 
# + . . 
# + # . 
# @ # * 
# . # . 

ITERAÇÃO: 2
['N', 'N', 'E', 'E', 'S', 'S']
Expandidos 9
# # # # # 
# + + @ . 
# + # # # 
# + # * . 
# . # . . 

ITERAÇÃO: 3
['E', 'E', 'S', 'S', 'W', 'W']
Expandidos 10
# # # # # # # 
# . . + + @ . 
# . # # # # # 
# . # * . . . 
# . # . . . . 

ITERAÇÃO: 4
['E', 'E', 'S', 'S', 'W', 'W', 'W', 'W']
Expandidos 14
# # # # # # # # # 
# . . . . + + @ . 
# . # # # # # # . 
# . # * . . . . . 
# . # . . . . . . 

ITERAÇÃO: 5
['E', 'S', 'S', 'W', 'W', 'W', 'W', 'W']
Expandidos 13
# # # # # # # # # # 
# . . . . . . + + # 
# . # # # # # # + # 
# . # @ + + + + + # 
# . # . . . . . . # 

FIM: total de expandidos: 53
```

### A função ***planear_adapt_online***

#### Exemplo 4
Vejamos o exemplo seguinte, em que basta apenas uma iteração. Nada muda neste exemplo em comparação com o Exemplo 1.
```python
pacman=(1,1)
pastilha=(1,6)
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
fronteira = quadro(0,0,10)
obstaculos=fronteira | l | c
planeia_adaptativo_online(pacman,pastilha,obstaculos)
```

dará como output:
```python
MUNDO
# # # # # # # # # # 
# @ . . . . . . . # 
# . # # # # # # . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# * # . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 

MODELO
# # # 
# @ . 
# . # 
. . . 
. . . 
. . . 
. * . 

ITERAÇÃO: 1
['S', 'S', 'S', 'S', 'S']
Expandidos 5
Novas heurísticas: [((1, 1), 5), ((1, 2), 4), ((1, 3), 3), ((1, 4), 2), ((1, 5), 1)]
# # # 
# + . 
# + # 
# + # 
# + # 
# + # 
# @ # 
# . . 

FIM: total de expandidos: 5
```

#### Exemplo 5

```python
pacman=(1,2)
pastilha=(3,6)
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
fronteira = quadro(0,0,10)
obstaculos=fronteira | l | c
planeia_adaptativo_online(pacman,pastilha,obstaculos)

MUNDO
# # # # # # # # # # 
# . . . . . . . . # 
# @ # # # # # # . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # * . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 

MODELO
# . . . 
# @ # . 
# . # . 
. . . . 
. . . . 
. . . * 

ITERAÇÃO: 1
['S', 'S', 'S', 'S', 'E', 'E']
Expandidos 10
Novas heurísticas: [((1, 2), 6), ((1, 3), 5), ((1, 4), 4), ((1, 5), 3), ((1, 6), 2), ((2, 4), 3), ((2, 5), 2), ((2, 6), 1), ((3, 4), 2), ((3, 5), 1)]
# . . . 
# + # . 
# + # . 
# + # . 
# + # . 
# @ # * 
# . . . 

ITERAÇÃO: 2
['S', 'E', 'E', 'N']
Expandidos 5
Novas heurísticas: [((1, 2), 6), ((1, 3), 5), ((1, 4), 4), ((1, 5), 3), ((1, 6), 4), ((1, 7), 3), ((2, 4), 3), ((2, 5), 2), ((2, 6), 1), ((2, 7), 2), ((3, 4), 2), ((3, 5), 1), ((3, 7), 1)]
# . . . 
# . # . 
# . # . 
# . # . 
# . # . 
# + # @ 
# + + + 
# . . . 

FIM: total de expandidos: 15
```
#### Exemplo 6  
**Explicação**  
Reparem na Iteração 1 e 2 abaixo. Na iteração 1, a heurística para a posição (1,2) é igual a 3, que se obtem do seguinte: o custo para chegar da posição inicial a (1,2) é igual a 1; o custo da solução é 4 (tamanho da solução); logo, a nova heurística é 4-1=3.  Este valor muda para 5 na iteração 2, que se obtem do seguinte: a distância da posição inicial a (1,2) continua a ser 1, e a distância da posição inicial à pastilha passou a ser 6 (tamanho da solução), logo 6-1=5.

```python
pacman=(1,1)
pastilha=(3,3)
l = line(2,2,1,0,6)
c = line(2,3,0,1,4)
fronteira = quadro(0,0,10)
obstaculos=fronteira | l | c
planeia_adaptativo_online(pacman,pastilha,obstaculos)

MUNDO
# # # # # # # # # # 
# @ . . . . . . . # 
# . # # # # # # . # 
# . # * . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . # . . . . . . # 
# . . . . . . . . # 
# . . . . . . . . # 
# # # # # # # # # # 

MODELO
# # # . 
# @ . . 
# . # . 
. . . * 

ITERAÇÃO: 1
['S', 'S', 'E', 'E']
Expandidos 7
Novas heurísticas: [((1, 1), 4), ((1, 2), 3), ((1, 3), 2), ((2, 1), 3), ((2, 3), 1), ((3, 1), 2), ((3, 2), 1)]
# # # . 
# + . . 
# + # . 
# @ # * 
# . # . 

ITERAÇÃO: 2
['N', 'N', 'E', 'E', 'S', 'S']
Expandidos 9
Novas heurísticas: [((1, 1), 4), ((1, 2), 5), ((1, 3), 6), ((1, 4), 5), ((1, 5), 4), ((2, 1), 3), ((2, 3), 1), ((2, 5), 3), ((3, 1), 2), ((3, 2), 1)]
# # # # # 
# + + @ . 
# + # # # 
# + # * . 
# . # . . 

ITERAÇÃO: 3
['E', 'E', 'S', 'S', 'W', 'W']
Expandidos 8
Novas heurísticas: [((1, 1), 4), ((1, 2), 5), ((1, 3), 6), ((1, 4), 5), ((1, 5), 4), ((2, 1), 5), ((2, 3), 1), ((2, 5), 3), ((3, 1), 6), ((3, 2), 1), ((4, 1), 5), ((4, 3), 1), ((5, 1), 4), ((5, 2), 3), ((5, 3), 2)]
# # # # # # # 
# . . + + @ . 
# . # # # # # 
# . # * . . . 
# . # . . . . 

ITERAÇÃO: 4
['E', 'E', 'S', 'S', 'W', 'W', 'W', 'W']
Expandidos 12
Novas heurísticas: [((1, 1), 4), ((1, 2), 5), ((1, 3), 6), ((1, 4), 5), ((1, 5), 4), ((2, 1), 5), ((2, 3), 1), ((2, 5), 3), ((3, 1), 6), ((3, 2), 1), ((4, 1), 7), ((4, 3), 1), ((5, 1), 8), ((5, 2), 3), ((5, 3), 2), ((6, 1), 7), ((6, 3), 3), ((7, 1), 6), ((7, 2), 5), ((7, 3), 4)]
# # # # # # # # # 
# . . . . + + @ . 
# . # # # # # # . 
# . # * . . . . . 
# . # . . . . . . 

ITERAÇÃO: 5
['E', 'S', 'S', 'W', 'W', 'W', 'W', 'W']
Expandidos 9
Novas heurísticas: [((1, 1), 4), ((1, 2), 5), ((1, 3), 6), ((1, 4), 5), ((1, 5), 4), ((2, 1), 5), ((2, 3), 1), ((2, 5), 3), ((3, 1), 6), ((3, 2), 1), ((4, 1), 7), ((4, 3), 1), ((5, 1), 8), ((5, 2), 3), ((5, 3), 2), ((6, 1), 7), ((6, 3), 3), ((7, 1), 8), ((7, 2), 5), ((7, 3), 4), ((8, 1), 7), ((8, 2), 6), ((8, 3), 5)]
# # # # # # # # # # 
# . . . . . . + + # 
# . # # # # # # + # 
# . # @ + + + + + # 
# . # . . . . . . # 

FIM: total de expandidos: 45
```


## Submissão

### Quiz
Cada grupo deve completar a implementação da função pedida e testá-la no link do *quiz* **Projecto 2: Pacman por Caminhos Desconhecidos** que está na página da disciplina, introduzindo aí o vosso código.

Esse *quiz* é constituído por duas únicas perguntas. A implementação das funções `planeia_online` e `planeia_adaptativo_online` serão avaliadas através de um conjunto de testes automáticos visíveis e mais alguns testes escondidos, valendo no total $1.25$ valores. A primeira função vale $0.95$, sendo que os testes visiveis valem $0.20$ e os invisíveis $0.75$. A segunda função vale $0.30$, sendo que os testes visíveis valem $0.05$ e os invisíveis $0.25$.

Este projecto é de correcção automatica e podem ir verificando o código fazendo check nos testes visíveis e invisíveis recebendo o feedback. Podem submeter as vezes que quiserem (independentemente do elemento do grupo), sendo a submissão com melhor nota a que será considerada.  
Não alterem as funções que estão no ficheiro --- `line`, `quadro`, `display`, e `manhattan`.


### Ficheiro Python
Simultaneamente é necessário submeter na página da disciplina o ficheiro python que contém todo o código utilizado para implementar as funções pedidas. Os grupos que não tiverem submetido o ficheiro python terão 0 valores de nota final neste projecto, independentemente do resultado da avaliação automática. São livres de implementar outras funções e poderão importar outros módulos desde que sejam reconhecidos pelo Moodle.

Esse ficheiro deve chamar-se *SI-proj2-XX.py* em que substituem XX pelo número do grupo. Basta um dos elementos submeter o ficheiro.

## Dúvidas
Todas as dúvidas que achem que são do interesse geral devem ser submetidas para o fórum da disciplina no Moodle. De resto, dúvidas mais privadas que envolvam resultados de testes escondidos e código, devem ser dirigidas apenas para nrgarcia@fc.ul.pt.